# CD
### 1. Write a program to search for a given pattern in a set of files. It should support regular expressions. It should work similar to grep and fgrep of Linux environment. 


### main.c
```
#include <stdio.h>
int main(){
    int x = 0;
    if(x)
    x--;
    else
    ++x;
    return 0;
}
```

In [64]:
import re

# s = input("Enter filename : ")
s = 'main.c'
f = open(s, 'r')
text = f.read()

symbols = ['!', '@', '#', '$', '%', '&', '^', '*']
oparators = ['+', '-', '*', '/', '=', '+=', '-=', '==', '<', '>', '<=', '>=']
keywords = ['auto','break', 'case', 'char', 'const', 'continue', 'default', 'do', 
			'double', 'else', 'enum', 'extern', 'float', 'for', 'goto', 'if', 
			'int', 'long', 'register', 'return', 'short', 'signed', 'sizeof', 'static', 
			'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while']
delimiters = [' ', '	', '.', ',', '\n', ';', '(', ')', '<', '>', '{', '}', '[', ']']


in_keywords = []
in_spl_symbols = []
in_oparators = []
in_delimiters = []
in_identifiers = []
in_constants = []

tokens = []
isStr = False
isWord = False
isCmt = 0
token = ''

for i in text:
	if i == '/':
		isCmt = isCmt+1

	elif isCmt == 2:
		if i == '\n':
			token = ''
			isCmt = 0
	
	elif i == '"' or i == "'":
		if isStr:
			tokens.append(token)
			token = ''
		isStr = not isStr 

	elif isStr:
		token = token+i
    
	elif i in symbols:
		tokens.append(i)
           
	elif i.isalnum() and not isWord:
		isWord = True
		token = i
    
	elif (i in delimiters) or (i in oparators):
		if token:
			tokens.append(token)
			token = ''
        
		if not (i==' ' or i=='\n' or i=='	'):
			tokens.append(i)

	elif isWord:
		token = token+i


for token in tokens:
	if token in symbols:
		in_spl_symbols.append(token)

	elif token in oparators:
		in_oparators.append(token)

	elif token in keywords:
		in_keywords.append(token)
				
	elif re.search("^[_a-zA-Z][_a-zA-Z0-9]*$",token):
		in_identifiers.append(token)
		
	elif token in delimiters:
		in_delimiters.append(token)
		
	else:
		in_constants.append(token)
	
						
print("No of tokens = ", len(tokens))   
print("\n\nNo. of keywords = ",len(in_keywords))
print("\nNo. of special symbols = ",len(in_spl_symbols))
print("\nNo. of oparators = ",len(in_oparators))
print("\nNo. of identifiers = ",len(in_identifiers))
print("\nNo. of constants = ",len(in_constants))
print("\nNo. of delimiters = ",len(in_delimiters))
f.close()   

No of tokens =  34

No. of keywords =  5
['int', 'int', 'if', 'else', 'return']

No. of special symbols =  1
['#']

No. of oparators =  7
['<', '>', '=', '-', '-', '+', '+']

No. of identifiers =  8
['include', 'stdio', 'h', 'main', 'x', 'x', 'x', 'x']

No. of constants =  2
['0', '0']

No. of delimiters =  11
['.', '(', ')', '{', ';', '(', ')', ';', ';', ';', '}']


### 2. Write programs to implement DFA and NFA.(Input : DFA or NFAand a string and Output : Verification of any given string for acceptance.) 

In [65]:
class DFA:
    def __init__(self,Q,S,D,q0,F) -> None:
        self.Q = Q
        self.S = S # Sigma - Alphabets
        self.D = D # Delta 
        self.s = q0
        self.F = F
    
    def check(self,inp):
        if inp == '' and self.s == 0 :
            return 'ACCEPTED'
        state = self.s
        for i in inp:
            if i not in self.S:
                return 'Rejected - Not an Alphabet'
            state = self.D[(state,i)]
        if state in self.F:
            return 'ACCEPTED'
        return 'Rejected - Not in Language'

class NFA(DFA):

    def check(self,inp):
        if inp == '' and self.s == 0 :
            return 'ACCEPTED'
        state = [self.s]
        for i in inp:
            if i not in self.S:
                return 'Rejected - Not an Alphabet'
            newstate = []
            for j in state:
                if (j,i) in self.D:
                    tempstate = self.D[(j,i)]
                    for k in tempstate:
                        newstate.append(k)
            state = list(set(newstate))
        for s in state:
            if s in self.F:
                return 'ACCEPTED'
        return 'Rejected - Not in Language'


dfa = DFA(3,('a','b'),{(0,'a'):1,(0,'b'):0,(1,'a'):2,(1,'b'):1,(2,'a'):0,(2,'b'):2},0,{0})
dfa.check('aababaaab')

'ACCEPTED'

In [66]:
dfa.check('aabbaa')

'Rejected - Not in Language'

In [67]:
nfa = NFA(2,('a','b'),{(0,'a'):[1],(1,'b'):[0]},0,{1})
nfa.check('abababa')

'ACCEPTED'

In [68]:
nfa.check('abab')

'Rejected - Not in Language'

### 3. Design a PDA for any given CNF. Simulate the processing of a string using the PDA and show the parse tree. 

In [69]:
class DPDA:
    
    def __init__(self, trf, input, state):
        
        self.head = 0
        self.trf = {}
        self.state = str(state)
        self.input = input
        self.trf = trf
        self.stack = ['Z']
        
    def step(self):
        
        a = self.input[self.head]
        s = self.stack.pop()
        state, ss = self.trf.get((self.state, a, s))
        if ss != 'ε':
            for s in ss[::-1]:
                self.stack.append(s)
        self.state = state
        print('{:20s} [{:10s}] {:5s}'.format(self.input[self.head:], 
                       ''.join(self.stack), self.state))        
        self.head += 1
    
    def run(self):
        
        print('{:20s} [{:10s}] {:5s}'.format(self.input[self.head:], 
                              ''.join(self.stack), self.state))
        
        while self.head  < len(self.input):
            self.step()

        s = self.stack.pop()        
        if self.trf.get((self.state, 'ε', s)):
            state, ss = self.trf.get((self.state, 'ε', s))
            self.state = state        
            print('{:20s} [{:10s}] {:5s}'.format('ε', 
                 ''.join(self.stack), self.state))
        if self.state == 'acc':
            return "Accepted"
        return "Rejected"
        
# run DPDA to accept the input string a^9b^9
DPDA({('q', 'a', 'Z'): ('q', 'XZ'),
     ('q', 'a', 'X'): ('q', 'XX'),
     ('q', 'b', 'X'): ('p', 'ε'),
     ('p', 'b', 'X'): ('p', 'ε'),
     ('p', 'ε', 'Z'): ('acc', 'Z'),
    }, 
    'aaaaabbbbb', 'q').run()

aaaaabbbbb           [Z         ] q    
aaaaabbbbb           [ZX        ] q    
aaaabbbbb            [ZXX       ] q    
aaabbbbb             [ZXXX      ] q    
aabbbbb              [ZXXXX     ] q    
abbbbb               [ZXXXXX    ] q    
bbbbb                [ZXXXX     ] p    
bbbb                 [ZXXX      ] p    
bbb                  [ZXX       ] p    
bb                   [ZX        ] p    
b                    [Z         ] p    
ε                    [          ] acc  


'Accepted'

In [70]:
Grammar = (('S','A','B'),('a','b'),{'S':['AB','BC'],'A':['BA','a'],'B':['CC','b'],'C':['AB','a']},'S') #VTPS

def CYK(Grammar,inp):
    n = len(inp)
    DP = [[[] for j in range(n)] for i in range(n)]
    for i in range(n):
        for k,v in Grammar[2].items():
            for r in v:
                if r == inp[i] :
                    DP[i][i].append(k)
    for i in range(2,n+1):
        for j in range(n-i+1):
            k = j+i-1
            for l in range(j,k-1):
                for prod,rules in Grammar[2].items():
                    for rule in rules:
                        if len(rule) > 1 and rule[0] == DP[j][l] and rule[1] == DP[l+1][k] :
                            DP[j][k].append(prod)
                            dis(DP,n)
def dis(DP,n):
    for i in range(n):
        for j in range(n):
            print(DP[i][j],end='\t\t')
        print()

    
CYK(Grammar , 'baaba')


In [71]:
non_terminals = ["S", "N", "D", "P",
				"V", "A"]
terminals = ["a","b", "c", "d",
			"e", "f",
			"g", "h","i"]

# Rules of the grammar
R = {
	"S": [["D", "N"]],
	"N": [["P", "N"], ["b"],
			["c"], ["d"]],
	"P": [["V", "A"], ["f"],
			["c"], ["e"]],
	"D": [["a"]],
	"V": [["g"], ["i"]],
	"A": [["f"], ["c"], ["e"],
		["h"]]
	}

def cykParse(inp):
	n = len(inp)
	T = [[set([]) for j in range(n)] for i in range(n)]

	for j in range(0, n):
		for lhs, rule in R.items():
			for rhs in rule:
				if len(rhs) == 1 and rhs[0] == inp[j]:
					T[j][j].add(lhs)

		for i in range(j, -1, -1):
			for k in range(i, j + 1):	
				for lhs, rule in R.items():
					for rhs in rule:
						if len(rhs) == 2 and rhs[0] in T[i][k] and rhs[1] in T[k + 1][j]:
							T[i][j].add(lhs)

	if len(T[0][n-1]) != 0:
		print("True")
	else:
		print("False")

inp = "a g f c b".split()
cykParse(inp)


True


### 4. Design a Lexical analyzer for identifying different types of tokens used in C language. Note: The reserved keywords such as if, else, class, structetc must be reported as invalid identifiers. C allows identifier names to begin with underscore character too. 

In [ ]:
import re

# s = input("Enter filename : ")
s = 'main.c'
f = open(s, 'r')
text = f.read()

symbols = ['!', '@', '#', '$', '%', '&', '^', '*']
oparators = ['+', '-', '*', '/', '=', '+=', '-=', '==', '<', '>', '<=', '>=']
keywords = ['auto','break', 'case', 'char', 'const', 'continue', 'default', 'do', 
			'double', 'else', 'enum', 'extern', 'float', 'for', 'goto', 'if', 
			'int', 'long', 'register', 'return', 'short', 'signed', 'sizeof', 'static', 
			'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while']
delimiters = [' ', '	', '.', ',', '\n', ';', '(', ')', '<', '>', '{', '}', '[', ']']


in_keywords = []
in_spl_symbols = []
in_oparators = []
in_delimiters = []
in_identifiers = []
in_constants = []

tokens = []
isStr = False
isWord = False
isCmt = 0
token = ''

for i in text:
	if i == '/':
		isCmt = isCmt+1

	elif isCmt == 2:
		if i == '\n':
			token = ''
			isCmt = 0
	
	elif i == '"' or i == "'":
		if isStr:
			tokens.append(token)
			token = ''
		isStr = not isStr 

	elif isStr:
		token = token+i
    
	elif i in symbols:
		tokens.append(i)
           
	elif i.isalnum() and not isWord:
		isWord = True
		token = i
    
	elif (i in delimiters) or (i in oparators):
		if token:
			tokens.append(token)
			token = ''
        
		if not (i==' ' or i=='\n' or i=='	'):
			tokens.append(i)

	elif isWord:
		token = token+i


for token in tokens:
	if token in symbols:
		in_spl_symbols.append(token)

	elif token in oparators:
		in_oparators.append(token)

	elif token in keywords:
		in_keywords.append(token)
				
	elif re.search("^[_a-zA-Z][_a-zA-Z0-9]*$",token):
		in_identifiers.append(token)
		
	elif token in delimiters:
		in_delimiters.append(token)
		
	else:
		in_constants.append(token)
	
						
print("No of tokens = ", len(tokens))   
print("\nNo. of keywords = ",len(in_keywords))
print(in_keywords);
print("\nNo. of special symbols = ",len(in_spl_symbols))
print(in_spl_symbols);
print("\nNo. of oparators = ",len(in_oparators))
print(in_oparators);
print("\nNo. of identifiers = ",len(in_identifiers))
print(in_identifiers);
print("\nNo. of constants = ",len(in_constants))
print(in_constants);
print("\nNo. of delimiters = ",len(in_delimiters))
print(in_delimiters);
f.close()   

5. Program to recognize the identifiers, if and switch statements of C using a lexical analyzer generator tool. 

### 6. Consider the following grammar: 
#### S --> ABC 
#### A-->abA | ab 
#### B--> b | BC 
#### C--> c | cC 
### Design any shift reduced parser which accepts a string and tells whether the string is accepted by above grammar or not. 

In [72]:
g = {'S': 'ABC', 'A': ['abA', 'ab'], 'B': ['b', 'BC'], 'C': ['c', 'cC']}
# G = (V,T,P,S)
G = (('S', 'A', 'B', 'C'), ('a', 'b', 'c', '$'), g, 'S')

Items = 11

pt_action = {
    (0, 'a'): 'S3',
    (1, '$'): 'ACC',
    (2, 'b'): 'S4',
    (3, 'b'): 'S6',
    (4, 'c'): 'R4',
    (5, 'c'): 'S8',
    (6, 'a'): 'S3',
    (6, 'b'): 'R3',
    (7, 'c'): 'R5',
    (7, '$'): 'R1',
    (8, 'c'): 'S8',
    (8, '$'): 'R6',
    (9, 'c'): 'R7',
    (9, '$'): 'R7',
    (10, 'b'): 'R2'
}
pt_goto = {
    (0, 'S'): 1,
    (0, 'A'): 2,
    (2, 'B'): 5,
    (5, 'C'): 7,
    (6, 'A'): 10,
    (8, 'C'): 9
}

R = {
    'R1': ['S', 'ABC'],
    'R2': ['A', 'abA'],
    'R3': ['A', 'ab'],
    'R4': ['B', 'b'],
    'R5': ['B', 'BC'],
    'R6': ['C', 'c'],
    'R7': ['C', 'cC']
}


def SRParser(s: str):
    s = s + '$'
    stack = ['$', 0]
    sptr = 0
    print('STACK\t\t\tISUT\t\t\tACTION')
    while(stack[-1] != 'ACC' or sptr < len(s)):
        if stack[-1] in range(Items):
            if s[sptr] in G[1]:
                if (stack[-1], s[sptr]) not in pt_action:
                    break
                act = pt_action[(stack[-1], s[sptr])]
                if 'S' in act:
                    stack.append(s[sptr])
                    stack.append(int(act[1]))
                    sptr += 1
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tSHIFT {act}')
                elif 'R' in act:
                    red_move = R[act]
                    popnum = len(red_move[1]) * 2
                    for i in range(popnum):
                        stack.pop()
                    stack.append(red_move[0])
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tREDUCE {act}')
                elif 'ACC' in act:
                    popnum = 2 * 2
                    for i in range(popnum):
                        stack.pop()
                    stack.append('ACC')
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tACCEPT {act}')
                else:
                    break
        elif stack[-1] in G[0]:
            stack.append(pt_goto[(stack[-2], stack[-1])])
            print(f'{stack}    \t\t{s[sptr:]}    \t\tGOTO {stack[-1]}')
        else:
            break
    if stack[0] == 'ACC':
        return "Accepted"
    return "Rejected"


SRParser('abbc'),SRParser('abbbcc')
#SRParser('abbcc'),SRParser('ababbcc')


STACK			ISUT			ACTION
['$', 0, 'a', 3]    		bbc$    		SHIFT S3
['$', 0, 'a', 3, 'b', 6]    		bc$    		SHIFT S6
['$', 0, 'A']    		bc$    		REDUCE R3
['$', 0, 'A', 2]    		bc$    		GOTO 2
['$', 0, 'A', 2, 'b', 4]    		c$    		SHIFT S4
['$', 0, 'A', 2, 'B']    		c$    		REDUCE R4
['$', 0, 'A', 2, 'B', 5]    		c$    		GOTO 5
['$', 0, 'A', 2, 'B', 5, 'c', 8]    		$    		SHIFT S8
['$', 0, 'A', 2, 'B', 5, 'C']    		$    		REDUCE R6
['$', 0, 'A', 2, 'B', 5, 'C', 7]    		$    		GOTO 7
['$', 0, 'S']    		$    		REDUCE R1
['$', 0, 'S', 1]    		$    		GOTO 1
['ACC']    		$    		ACCEPT ACC
STACK			ISUT			ACTION
['$', 0, 'a', 3]    		bbbcc$    		SHIFT S3
['$', 0, 'a', 3, 'b', 6]    		bbcc$    		SHIFT S6
['$', 0, 'A']    		bbcc$    		REDUCE R3
['$', 0, 'A', 2]    		bbcc$    		GOTO 2
['$', 0, 'A', 2, 'b', 4]    		bcc$    		SHIFT S4


('Accepted', 'Rejected')

7. YACC program that reads the input expression and convert it to post fix expression. 

# CNS
### 1. Working with Sniffers for monitoring network communication (Ethereal) 

https://www.lifewire.com/wireshark-tutorial-4143298

##### What Is Wireshark?
Originally known as Ethereal, Wireshark displays data from hundreds of different protocols on all major network types. Data packets can be viewed in real-time or analyzed offline. Wireshark supports dozens of capture/trace file formats, including CAP and ERF. Integrated decryption tools display the encrypted packets for several common protocols, including WEP and WPA/WPA2.

##### How to Download and Install Wireshark
Wireshark can be downloaded at no cost from the Wireshark Foundation website for both macOS and Windows. You'll see the latest stable release and the current developmental release. Unless you're an advanced user, download the stable version.

Wireshark can be downloaded at no cost from the Wireshark Foundation website for both macOS and Windows operating systems.
During the Windows setup process, choose to install WinPcap or Npcap if prompted as these include libraries required for live data capture.


You must be logged in to the device as an administrator to use Wireshark. In Windows 10, search for Wireshark and select Run as administrator. In macOS, right-click the app icon and select Get Info. In the Sharing & Permissions settings, give the admin Read & Write privileges.


The application is also available for Linux and other UNIX-like platforms including Red Hat, Solaris, and FreeBSD. The binaries required for these operating systems can be found toward the bottom of the Wireshark download page under the Third-Party Packages section. You can also download Wireshark's source code from this page.

##### How to Capture Data Packets With Wireshark
When you launch Wireshark, a welcome screen lists the available network connections on your current device. Displayed to the right of each is an EKG-style line graph that represents live traffic on that network.

##### To begin capturing packets with Wireshark:

Select one or more of networks, go to the menu bar, then select Capture.

To select multiple networks, hold the Shift key as you make your selection.


In the Wireshark Capture Interfaces window, select Start.

There are other ways to initiate packet capturing. Select the shark fin on the left side of the Wireshark toolbar, press ​Ctrl+E, or double-click the network.


Select File > Save As or choose an Export option to record the capture.


To stop capturing, press Ctrl+E. Or, go to the Wireshark toolbar and select the red Stop button that's located next to the shark fin.


##### How to View and Analyze Packet Contents
The captured data interface contains three main sections:

The packet list pane (the top section)
The packet details pane (the middle section)
The packet bytes pane (the bottom section)
The captured data interface contains three main sections: the packet list pane (the top section); the packet details pane (the middle section); and the packet bytes pane (the bottom section).
Packet List
The packet list pane, located at the top of the window, shows all packets found in the active capture file. Each packet has its own row and corresponding number assigned to it, along with each of these data points:

No: This field indicates which packets are part of the same conversation. It remains blank until you select a packet.
Time: The timestamp of when the packet was captured is displayed in this column. The default format is the number of seconds or partial seconds since this specific capture file was first created.
Source: This column contains the address (IP or other) where the packet originated.
Destination: This column contains the address that the packet is being sent to.
Protocol: The packet's protocol name, such as TCP, can be found in this column.
Length: The packet length, in bytes, is displayed in this column.
Info: Additional details about the packet are presented here. The contents of this column can vary greatly depending on packet contents.
To change the time format to something more useful (such as the actual time of day), select View > Time Display Format.


When a packet is selected in the top pane, you may notice one or more symbols appear in the No. column. Open or closed brackets and a straight horizontal line indicate whether a packet or group of packets are part of the same back-and-forth conversation on the network. A broken horizontal line signifies that a packet is not part of the conversation.


##### Packet Details
The details pane, found in the middle, presents the protocols and protocol fields of the selected packet in a collapsible format. In addition to expanding each selection, you can apply individual Wireshark filters based on specific details and follow streams of data based on protocol type by right-clicking the desired item.


##### Packet Bytes
At the bottom is the packet bytes pane, which displays the raw data of the selected packet in a hexadecimal view. This hex dump contains 16 hexadecimal bytes and 16 ASCII bytes alongside the data offset.

Selecting a specific portion of this data automatically highlights its corresponding section in the packet details pane and vice versa. Any bytes that cannot be printed are represented by a period.


To display this data in bit format as opposed to hexadecimal, right-click anywhere within the pane and select as bits.


##### How to Use Wireshark Filters
Capture filters instruct Wireshark to only record packets that meet specified criteria. Filters can also be applied to a capture file that has been created so that only certain packets are shown. These are referred to as display filters.

Wireshark provides a large number of predefined filters by default. To use one of these existing filters, enter its name in the Apply a display filter entry field located below the Wireshark toolbar or in the Enter a capture filter field located in the center of the welcome screen.

For example, if you want to display TCP packets, type tcp. The Wireshark autocomplete feature shows suggested names as you begin typing, making it easier to find the correct moniker for the filter you're seeking.


Another way to choose a filter is to select the bookmark on the left side of the entry field. Choose Manage Filter Expressions or Manage Display Filters to add, remove, or edit filters.


You can also access previously used filters by selecting the down arrow on the right side of the entry field to display a history drop-down list.


Capture filters are applied as soon as you begin recording network traffic. To apply a display filter, select the right arrow on the right side of the entry field.

Wireshark Color Rules
While Wireshark's capture and display filters limit which packets are recorded or shown on the screen, its colorization function takes things a step further: It can distinguish between different packet types based on their individual hue. This quickly locates certain packets within a saved set by their row color in the packet list pane.

Wireshark coloring rules dialog opened in front of main Wireshark window
Wireshark comes with about 20 default coloring rules, each can be edited, disabled, or deleted. Select View > Coloring Rules for an overview of what each color means. You can also add your own color-based filters.


Select View > Colorize Packet List to toggle packet colorization on and off.

##### Statistics in Wireshark
Other useful metrics are available through the Statistics drop-down menu. These include size and timing information about the capture file, along with dozens of charts and graphs ranging in topic from packet conversation breakdowns to load distribution of HTTP requests.

Several other useful metrics are available via the Statistics drop-down menu found toward the top of the screen.
Display filters can be applied to many of these statistics via their interfaces, and the results can be exported to common file formats, including CSV, XML, and TXT.

Wireshark Advanced Features
Wireshark also supports advanced features, including the ability to write protocol dissectors in the Lua programming language.


### 2. Understanding of cryptographic algorithms and implementation of the same in C or C++ 

##### CEASER

In [73]:
def encrypt(text,s):
       
    # Cipher(n) = De-cipher(26-n)
    s=s   
    text =text.replace(" ","")
    result=""  #empty string
    for i in range(len(text)):
        char=text[i]
        if(char.isupper()):  #if the text[i] is in upper case
            result=result+chr((ord(char)+s-65)%26+65)
        else:
            result=result+chr((ord(char)+s-97)%26+97)
    return result


# word=str(input("enter the word:"))
word = "Hello World"
# k=int(input("Enter the key: "))
k = 3

print("Encoded word in Caeser cipher is: ",encrypt(word,k))

Encoded word in Caeser cipher is:  KhoorZruog


In [74]:
def decrypt(text,s):
    
    # Cipher(n) = De-cipher(26-n)
    s=26-s 
        
    result=""  #empty string
    for i in range(len(text)):
        char=text[i]
        if(char.isupper()):  #if the text[i] is in upper case
            result=result+chr((ord(char)+s-65)%26+65)
        else:
            result=result+chr((ord(char)+s-97)%26+97)
    return result


# word=str(input("enter the word:"))
word="KhoorZruog"
d = 3

print("Encoded word in Caeser cipher is: ",decrypt(word,d))

Encoded word in Caeser cipher is:  HelloWorld


##### PLAYFAIR

In [75]:

import string


def key_generation(key):
    main = string.ascii_lowercase.replace('j', '.')
    key = key.lower()
    key_matrix = ['' for i in range(5)]
    i = 0
    j = 0
    for c in key:
        if c in main:
            key_matrix[i] += c
            main = main.replace(c, '.')
            j += 1
            if(j > 4):
                i += 1
                j = 0
    for c in main:
        if c != '.':
            key_matrix[i] += c
            j += 1
            if j > 4:
                i += 1
                j = 0
    return(key_matrix)


def conversion(plain_text):
    plain_text_pairs = []
    cipher_text_pairs = []
    plain_text = plain_text.replace(" ", "")
    plain_text = plain_text.lower()
    i = 0
    while i < len(plain_text):
        a = plain_text[i]
        b = ''
        if((i+1) == len(plain_text)):
            b = 'x'
        else:
            b = plain_text[i+1]
        if(a != b):
            plain_text_pairs.append(a+b)
            i += 2
        else:
            plain_text_pairs.append(a+'x')
            i += 1
    print("plain text pairs: ", plain_text_pairs)

    for pair in plain_text_pairs:
        flag = False
        for row in key_matrix:
            if(pair[0] in row and pair[1] in row):
                j0 = row.find(pair[0])
                j1 = row.find(pair[1])
                cipher_text_pair = row[(j0+1) % 5]+row[(j1+1) % 5]
                cipher_text_pairs.append(cipher_text_pair)
                flag = True
        if flag:
            continue
        for j in range(5):
            col = "".join([key_matrix[i][j] for i in range(5)])
            if(pair[0] in col and pair[1] in col):
                i0 = col.find(pair[0])
                i1 = col.find(pair[1])
                cipher_text_pair = col[(i0+1) % 5]+col[(i1+1) % 5]
                cipher_text_pairs.append(cipher_text_pair)
                flag = True
        if flag:
            continue

        i0 = 0
        i1 = 0
        j0 = 0
        j1 = 0

        for i in range(5):
            row = key_matrix[i]
            if(pair[0] in row):
                i0 = i
                j0 = row.find(pair[0])
            if(pair[1] in row):
                i1 = i
                j1 = row.find(pair[1])
        cipher_text_pair = key_matrix[i0][j1]+key_matrix[i1][j0]
        cipher_text_pairs.append(cipher_text_pair)

    print("cipher text pairs: ", cipher_text_pairs)
    print('plain text: ', plain_text)
    print('cipher text: ', "".join(cipher_text_pairs))


# key = input("Enter the key: ")
# key_matrix = key_generation(key)
# print("Key Matrix for encryption:")
# print(key_matrix)
# plain_text = input("Enter the message: ")

# conversion(plain_text)

'''
----------OUTPUT----------
Enter the key: Make my day beautiful
Key Matrix for encryption:
['makey', 'dbuti', 'flcgh', 'nopqr', 'svwxz']
Enter the message: hi there my name is abhiram
plain text pairs:  ['hi', 'th', 'er', 'em', 'yn', 'am', 'ei', 'sa', 'bh', 'ir', 'am']
cipher text pairs:  ['rh', 'ig', 'yq', 'ya', 'mr', 'ka', 'yt', 'vm', 'il', 'hz', 'ka']
plain text:  hitheremynameisabhiram
cipher text:  rhigyqyamrkaytvmilhzka
>>> 
'''


def conversion(cipher_text):
    plain_text_pairs = []
    cipher_text_pairs = []
    cipiher_text = cipher_text.lower()
    i = 0
    while i < len(cipher_text):
        a = cipher_text[i]
        b = cipher_text[i+1]
        cipher_text_pairs.append(a+b)
        i += 2
    print("cipher text pairs: ", cipher_text_pairs)
    for pair in cipher_text_pairs:
        flag = False
        for row in key_matrix:
            if(pair[0] in row and pair[1] in row):
                j0 = row.find(pair[0])
                j1 = row.find(pair[1])
                plain_text_pair = row[(j0+4) % 5]+row[(j1+4) % 5]
                plain_text_pairs.append(plain_text_pair)
                flag = True
        if flag:
            continue
        for j in range(5):
            col = "".join([key_matrix[i][j] for i in range(5)])
            if(pair[0] in col and pair[1] in col):
                i0 = col.find(pair[0])
                i1 = col.find(pair[1])
                plain_text_pair = col[(i0+4) % 5]+col[(i1+4) % 5]
                plain_text_pairs.append(plain_text_pair)
                flag = True
        if flag:
            continue
        i0 = 0
        i1 = 0
        j0 = 0
        j1 = 0
        for i in range(5):
            row = key_matrix[i]
            if(pair[0] in row):
                i0 = i
                j0 = row.find(pair[0])
            if(pair[1] in row):
                i1 = i
                j1 = row.find(pair[1])
        plain_text_pair = key_matrix[i0][j1]+key_matrix[i1][j0]
        plain_text_pairs.append(plain_text_pair)

    print("plain text pairs: ", plain_text_pairs)
    print('cipher text: ', "".join(cipher_text_pairs))
    print('plain text (message): ', "".join(plain_text_pairs))


# key = input("Enter the key: ")

# key_matrix = key_generation(key)
# print("Key Matrix for encryption:")
# print(key_matrix)
# cipher_text = input("Enter the encrypted message: ")

# conversion(cipher_text)

'''
----------OUTPUT----------
Enter the key: Make my day beautiful
Key Matrix for encryption:
['makey', 'dbuti', 'flcgh', 'nopqr', 'svwxz']
Enter the encrypted message: rhigyqyamrkaytvmilhzka
cipher text pairs:  ['rh', 'ig', 'yq', 'ya', 'mr', 'ka', 'yt', 'vm', 'il', 'hz', 'ka']
plain text pairs:  ['hi', 'th', 'er', 'em', 'yn', 'am', 'ei', 'sa', 'bh', 'ir', 'am']
cipher text:  rhigyqyamrkaytvmilhzka
plain text (message):  hitheremynameisabhiram
>>> 
'''



"\n----------OUTPUT----------\nEnter the key: Make my day beautiful\nKey Matrix for encryption:\n['makey', 'dbuti', 'flcgh', 'nopqr', 'svwxz']\nEnter the encrypted message: rhigyqyamrkaytvmilhzka\ncipher text pairs:  ['rh', 'ig', 'yq', 'ya', 'mr', 'ka', 'yt', 'vm', 'il', 'hz', 'ka']\nplain text pairs:  ['hi', 'th', 'er', 'em', 'yn', 'am', 'ei', 'sa', 'bh', 'ir', 'am']\ncipher text:  rhigyqyamrkaytvmilhzka\nplain text (message):  hitheremynameisabhiram\n>>> \n"

##### HILL

In [76]:
import string


main=string.ascii_lowercase

def generate_key(n,s):
    s=s.replace(" ","")
    s=s.lower()
    
    key_matrix=['' for i in range(n)]
    i=0;j=0
    for c in s:
        if c in main:
            key_matrix[i]+=c
            j+=1
            if(j>n-1):
                i+=1
                j=0
    print("The key matrix "+"("+str(n)+'x'+str(n)+") is:")
    print(key_matrix)
    
    key_num_matrix=[]
    for i in key_matrix:
        sub_array=[]
        for j in range(n):
            sub_array.append(ord(i[j])-ord('a'))
        key_num_matrix.append(sub_array)
        
    for i in key_num_matrix:
        print(i)
    return(key_num_matrix)
    

def message_matrix(s,n):
    s=s.replace(" ","")
    s=s.lower()
    final_matrix=[]
    if(len(s)%n!=0):

        while(len(s)%n!=0):
            s=s+'z'
    print("Converted plain_text for encryption: ",s)
    for k in range(len(s)//n):
        message_matrix=[]
        for i in range(n):
            sub=[]
            for j in range(1):
                sub.append(ord(s[i+(n*k)])-ord('a'))
            message_matrix.append(sub)
        final_matrix.append(message_matrix)
    print("The column matrices of plain text in numbers are:  ")
    for i in final_matrix:
        print(i)
    return(final_matrix)

def getCofactor(mat, temp, p, q, n): 
    i = 0
    j = 0
  
    for row in range(n):  
        for col in range(n): 
            if (row != p and col != q) : 
                temp[i][j] = mat[row][col] 
                j += 1        
                if (j == n - 1): 
                    j = 0
                    i += 1

def determinantOfMatrix(mat, n): 
  
    if (n == 1): 
        return mat[0][0] 
          
    temp = [[0 for x in range(n)]  
               for y in range(n)]  
    for f in range(n): 
        getCofactor(mat, temp, 0, f, n) 
        D += (sign * mat[0][f] *
              determinantOfMatrix(temp, n - 1))
        sign = -sign 
    return D 
  
def isInvertible(mat, n): 
    if (determinantOfMatrix(mat, n) != 0): 
        return True
    else: 
        return False


def multiply_and_convert(key,message):
    
    res_num = [[0 for x in range(len(message[0]))] for y in range(len(key))]
    
    for i in range(len(key)): 
        for j in range(len(message[0])):
            for k in range(len(message)): 
        
                res_num[i][j]+=key[i][k] * message[k][j]

    res_alpha = [['' for x in range(len(message[0]))] for y in range(len(key))]
    for i in range(len(key)):
        for j in range(len(message[0])):
    
            res_alpha[i][j]+=chr((res_num[i][j]%26)+97)
            
    return(res_alpha)
    
# n=int(input("What will be the order of square matrix: "))
# s=input("Enter the key: ")
# key=generate_key(n,s)
# if (isInvertible(key, len(key))): 
#     print("Yes it is invertable and can be decrypted") 
# else: 
#     print("No it is not invertable and cannot be decrypted")
    
# plain_text=input("Enter the message: ")
# message=message_matrix(plain_text,n)
# final_message=''
# for i in message:
#     sub=multiply_and_convert(key,i)
#     for j in sub:
#         for k in j:
#             final_message+=k
# print("plain message: ",plain_text)
# print("final encrypted message: ",final_message)

'''
----------OUTPUT----------
What will be the order of square matrix: 3
Enter the key: BACK UP ABC
The key matrix (3x3) is:
['bac', 'kup', 'abc']
[1, 0, 2]
[10, 20, 15]
[0, 1, 2]
Yes it is invertable and can be decrypted
Enter the message: hi there my name is abhiram
Converted plain_text for encryption:  hitheremynameisabhiramzz
The column matrices of plain text in numbers are:  
[[7], [8], [19]]
[[7], [4], [17]]
[[4], [12], [24]]
[[13], [0], [12]]
[[4], [8], [18]]
[[0], [1], [7]]
[[8], [17], [0]]
[[12], [25], [25]]
plain message:  hi there my name is abhiram
final encrypted message:  tvuppmaqilyyocsovpierkhx
>>> 
'''
import string
import numpy as np

main=string.ascii_lowercase

def generate_key(n,s):
    s=s.replace(" ","")
    s=s.lower()
    
    key_matrix=['' for i in range(n)]
    i=0;j=0
    for c in s:
        if c in main:
            key_matrix[i]+=c
            j+=1
            if(j>n-1):
                i+=1
                j=0
    print("The key matrix "+"("+str(n)+'x'+str(n)+") is:")
    print(key_matrix)
    
    key_num_matrix=[]
    for i in key_matrix:
        sub_array=[]
        for j in range(n):
            sub_array.append(ord(i[j])-ord('a'))
        key_num_matrix.append(sub_array)

    for i in key_num_matrix:
        print(i)
    return(key_num_matrix)


def modInverse(a, m) : 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

def method(a, m) :
    if(a>0):
        return (a%m)
    else:
        k=(abs(a)//m)+1
    return method(a+k*m,m)


def message_matrix(s,n):
    s=s.replace(" ","")
    s=s.lower()
    final_matrix=[]
    if(len(s)%n!=0):
        for i in range(abs(len(s)%n)):
            s=s+'z'
    print("Converted cipher_text for decryption: ",s)
    for k in range(len(s)//n):
        message_matrix=[]
        for i in range(n):
            sub=[]
            for j in range(1):
                sub.append(ord(s[i+(n*k)])-ord('a'))
            message_matrix.append(sub)
        final_matrix.append(message_matrix)
    print("The column matrices of plain text in numbers are:  ")
    for i in final_matrix:
        print(i)
    return(final_matrix)


def multiply_and_convert(key,message):
    
    res_num = [[0 for x in range(len(message[0]))] for y in range(len(key))]
    
    for i in range(len(key)): 
        for j in range(len(message[0])):
            for k in range(len(message)): 
                res_num[i][j]+=key[i][k] * message[k][j]

    res_alpha = [['' for x in range(len(message[0]))] for y in range(len(key))]
    for i in range(len(key)):
        for j in range(len(message[0])):
            res_alpha[i][j]+=chr((res_num[i][j]%26)+97)
            
    return(res_alpha)



# n=int(input("What will be the order of square matrix: "))
# s=input("Enter the key: ")
# key_matrix=generate_key(n,s)
# A = np.array(key_matrix)
# det=np.linalg.det(A)
# adjoint=det*np.linalg.inv(A)

# if(det!=0):
#     convert_det=modInverse(int(det),26)
#     adjoint=adjoint.tolist()
#     print("Adjoint Matrix before modulo26 operation: ")
#     for i in adjoint:
#         print(i)
#     print(convert_det)

#     for i in range(len(adjoint)):
#         for j in range(len(adjoint[i])):
#             adjoint[i][j]=round(adjoint[i][j])
#             adjoint[i][j]=method(adjoint[i][j],26)
#     print("Adjoint Matrix after modulo26 operation: ")
#     for i in adjoint:
#         print(i)

#     adjoint=np.array(adjoint)
#     inverse=convert_det*adjoint

#     inverse=inverse.tolist()
#     for i in range(len(inverse)):
#         for j in range(len(inverse[i])):
#             inverse[i][j]=inverse[i][j]%26

#     print("Inverse matrix after applying modulo26 operation: ")
#     for i in inverse:
#         print(i)

#     cipher_text=input("Enter the cipher text: ")
#     message=message_matrix(cipher_text,n)
#     plain_text=''
#     for i in message:
#         sub=multiply_and_convert(inverse,i)
#         for j in sub:
#             for k in j:
#                 plain_text+=k
                
#     print("plain message: ",plain_text)
# else:
#     print("Matrix cannot be inverted")

'''
----------OUTPUT----------
What will be the order of square matrix: 3
Enter the key: BACK UP ABC
The key matrix (3x3) is:
['bac', 'kup', 'abc']
[1, 0, 2]
[10, 20, 15]
[0, 1, 2]
Adjoint Matrix before modulo26 operation: 
[25.000000000000014, 2.000000000000001, -40.000000000000014]
[-20.000000000000004, 2.000000000000001, 5.000000000000001]
[10.000000000000002, -1.0000000000000004, 20.000000000000004]
11
Adjoint Matrix after modulo26 operation: 
[25, 2, 12]
[6, 2, 5]
[10, 25, 20]
Inverse matrix after applying modulo26 operation: 
[15, 22, 2]
[14, 22, 3]
[6, 15, 12]
Enter the cipher text: tvuppmaqilyyocsovpierkhx
Converted cipher_text for decryption:  tvuppmaqilyyocsovpierkhx
The column matrices of plain text in numbers are:  
[[19], [21], [20]]
[[15], [15], [12]]
[[0], [16], [8]]
[[11], [24], [24]]
[[14], [2], [18]]
[[14], [21], [15]]
[[8], [4], [17]]
[[10], [7], [23]]
plain message:  hitheremynameisabhiramzz
>>> 
'''
             
            

"\n----------OUTPUT----------\nWhat will be the order of square matrix: 3\nEnter the key: BACK UP ABC\nThe key matrix (3x3) is:\n['bac', 'kup', 'abc']\n[1, 0, 2]\n[10, 20, 15]\n[0, 1, 2]\nAdjoint Matrix before modulo26 operation: \n[25.000000000000014, 2.000000000000001, -40.000000000000014]\n[-20.000000000000004, 2.000000000000001, 5.000000000000001]\n[10.000000000000002, -1.0000000000000004, 20.000000000000004]\n11\nAdjoint Matrix after modulo26 operation: \n[25, 2, 12]\n[6, 2, 5]\n[10, 25, 20]\nInverse matrix after applying modulo26 operation: \n[15, 22, 2]\n[14, 22, 3]\n[6, 15, 12]\nEnter the cipher text: tvuppmaqilyyocsovpierkhx\nConverted cipher_text for decryption:  tvuppmaqilyyocsovpierkhx\nThe column matrices of plain text in numbers are:  \n[[19], [21], [20]]\n[[15], [15], [12]]\n[[0], [16], [8]]\n[[11], [24], [24]]\n[[14], [2], [18]]\n[[14], [21], [15]]\n[[8], [4], [17]]\n[[10], [7], [23]]\nplain message:  hitheremynameisabhiramzz\n>>> \n"

##### RAIL FENCE

In [77]:

def sequence(n):
    arr=[]
    i=0
    while(i<n-1):
        arr.append(i)
        i+=1
    while(i>0):
        arr.append(i)
        i-=1
    return(arr)
def railfence(s,n):
    s=s.lower()


    L=sequence(n)
    print("The raw sequence of indices: ",L)

    temp=L
    
    while(len(s)>len(L)):
        L=L+temp

    for i in range(len(L)-len(s)):
        L.pop()
    print("The row indices of the characters in the given string: ",L)
    
    
    print("Transformed message for encryption: ",s)

    num=0
    cipher_text=""
    while(num<n):
        for i in range(L.count(num)):
            cipher_text=cipher_text+s[L.index(num)]
            L[L.index(num)]=n
        num+=1
    print("The cipher text is: ",cipher_text)
   
# plain_text=input("Enter the string to be encrypted: ")
# n=int(input("Enter the number of rails: "))
# railfence(plain_text,n)

'''
----------OUTPUT----------
Enter the string to be encrypted: Hey everyone have good day
Enter the number of rails: 5
The raw sequence of indices:  [0, 1, 2, 3, 4, 3, 2, 1]
The row indices of the characters in the given string:  [0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1]
Transformed message for encryption:  hey everyone have good day
The cipher text is:  hyeaerov dyyenag  vehode o
>>>
'''
def sequence(n):
    arr=[]
    i=0
    while(i<n-1):
        arr.append(i)
        i+=1
    while(i>0):
        arr.append(i)
        i-=1
    return(arr)
def railfence(cipher_text,n):
    cipher_text=cipher_text.lower()


    L=sequence(n)
    print("The raw sequence of indices: ",L)

    temp=L
    
    while(len(cipher_text)>len(L)):
        L=L+temp

    for i in range(len(L)-len(cipher_text)):
        L.pop()
        
    temp1=sorted(L)
    
    print("The row indices of the characters in the cipher string: ",L)

    print("The row indices of the characters in the plain string: ",temp1)
    
    print("Transformed message for decryption: ",cipher_text)

    plain_text=""
    for i in L:
        k=temp1.index(i)
        temp1[k]=n
        plain_text+=cipher_text[k]
        
    print("The cipher text is: ",plain_text)


# cipher_text=input("Enter the string to be decrypted: ")
# n=int(input("Enter the number of rails: "))
# railfence(cipher_text,n)

'''
----------OUTPUT----------
Enter the string to be decrypted: hyeaerov dyyenag  vehode o
Enter the number of rails: 5
The raw sequence of indices:  [0, 1, 2, 3, 4, 3, 2, 1]
The row indices of the characters in the cipher string:  [0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1]
The row indices of the characters in the plain string:  [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4]
Transformed message for decryption:  hyeaerov dyyenag  vehode o
The cipher text is:  hey everyone have good day
>>> 
'''


'\n----------OUTPUT----------\nEnter the string to be decrypted: hyeaerov dyyenag  vehode o\nEnter the number of rails: 5\nThe raw sequence of indices:  [0, 1, 2, 3, 4, 3, 2, 1]\nThe row indices of the characters in the cipher string:  [0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1]\nThe row indices of the characters in the plain string:  [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4]\nTransformed message for decryption:  hyeaerov dyyenag  vehode o\nThe cipher text is:  hey everyone have good day\n>>> \n'

##### COLUMNAR

In [78]:
import math

def row(s,key):
    
    temp=[]
    for i in key:
        if i not in temp:
            temp.append(i)
    k=""
    for i in temp:
        k+=i
    print("The key used for encryption is: ",k)
    
    b=math.ceil(len(s)/len(k))
    
    if(b<len(k)):
        b=b+(len(k)-b)
    
    arr=[['_' for i in range(len(k))]
         for j in range(b)]
    i=0
    j=0
    for h in range(len(s)):
        arr[i][j]=s[h]
        j+=1
        if(j>len(k)-1):
            j=0
            i+=1
    print("The message matrix is: ")
    for i in arr:
        print(i)

    cipher_text=""
    kk=sorted(k)
    
    for i in kk:

        h=k.index(i)
        for j in range(len(arr)):
            cipher_text+=arr[j][h]
    print("The cipher text is: ",cipher_text)
        
# msg=input("Enter the message: ")
# key=input("Enter the key in alphabets: ")
# row(msg,key)

'''
----------OUTPUT----------
Enter the message: My computer is owned by me
Enter the key in alphabets: expensive
The key used for encryption is:  expnsiv
The message matrix is: 
['M', 'y', ' ', 'c', 'o', 'm', 'p']
['u', 't', 'e', 'r', ' ', 'i', 's']
[' ', 'o', 'w', 'n', 'e', 'd', ' ']
['b', 'y', ' ', 'm', 'e', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
The cipher text is:  Mu b___mid____crnm___ ew ___o ee___ps ____ytoy___
>>>
'''
import math

def row(s,key):

    temp=[]
    for i in key:
        if i not in temp:
            temp.append(i)
    k=""
    for i in temp:
        k+=i
    print("The key used for encryption is: ",k)
    
    arr=[['' for i in range(len(k))]
         for j in range(int(len(s)/len(k)))]

    kk=sorted(k)
    
    d=0
    for i in kk:
        h=k.index(i)
        for j in range(len(k)):
            arr[j][h]=s[d]
            d+=1
                
    print("The message matrix is: ")
    for i in arr:
        print(i)

    plain_text=""
    for i in arr:
        for j in i:
            plain_text+=j
    print("The plain text is: ",plain_text)

        
# msg=input("Enter the message to be decrypted: ")
# key=input("Enter the key in alphabets: ")
# row(msg,key)

'''
----------OUTPUT----------
Enter the message to be decrypted: Mu b___mid____crnm___ ew ___o ee___ps ____ytoy___
Enter the key in alphabets: expensive
The key used for encryption is:  expnsiv
The message matrix is: 
['M', 'y', ' ', 'c', 'o', 'm', 'p']
['u', 't', 'e', 'r', ' ', 'i', 's']
[' ', 'o', 'w', 'n', 'e', 'd', ' ']
['b', 'y', ' ', 'm', 'e', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
The plain text is:  My computer is owned by me_______________________
>>>
'''

"\n----------OUTPUT----------\nEnter the message to be decrypted: Mu b___mid____crnm___ ew ___o ee___ps ____ytoy___\nEnter the key in alphabets: expensive\nThe key used for encryption is:  expnsiv\nThe message matrix is: \n['M', 'y', ' ', 'c', 'o', 'm', 'p']\n['u', 't', 'e', 'r', ' ', 'i', 's']\n[' ', 'o', 'w', 'n', 'e', 'd', ' ']\n['b', 'y', ' ', 'm', 'e', '_', '_']\n['_', '_', '_', '_', '_', '_', '_']\n['_', '_', '_', '_', '_', '_', '_']\n['_', '_', '_', '_', '_', '_', '_']\nThe plain text is:  My computer is owned by me_______________________\n>>>\n"

##### RSA

In [79]:
import math
import string

main=string.ascii_lowercase

'''
function gcd(a, b)
    if b = 0
        return a
    else
        return gcd(b, a mod b)
'''

def multiplicative_inverse(a, m):
    a=a%m; 
    for x in range(1,m) : 
        if((a*x)%m==1) : 
            return x 
    return 1

'''
def get_mod_expo(base,exponent,modulus):
    result=1
    while exponent:
        d=exponent%2
        exponent=exponent//2
        if d:
            result=result*base%modulus
        base=base*base%modulus
    return result
'''

def generate_keypair(p, q):
    n=p*q
    print("Value of n: ",n)


    phi = (p-1)*(q-1)
    print("Value of phi(n): ", phi)



    print("Enter e such that is co-prime to ", phi,": ")
    e=int(input())



    g=math.gcd(e,phi)
    while(g!=1):
        print("The number you entered is not co-prime")
        e=int(input())
        g=math.gcd(e,phi)
        
    print("Value of exponent(e) entered is: ", e)


    d = multiplicative_inverse(e, phi)


    

    return (e,n),(d,n)

def encrypt(public_key, to_encrypt):
    key, n = public_key


    cipher=pow(to_encrypt,key)%n
    return cipher


def decrypt(private_key, to_decrypt):
    key, n = private_key


    decrypted=pow(to_decrypt,key)%n
    return decrypted

# p=int(input("Enter prime p: "))
# q=int(input("Enter prime q (!=p): "))

# while(p==q):
#     p=int(input("Enter prime p: "))
#     q=int(input("Enter prime q (!=p): "))
    
# print("Prime number p: ",p)
# print("Prime number q: ",q)

# print("Generating Public/Private key-pairs!")
# public, private = generate_keypair(p, q)
# print("Your public key is (e,n) ", public)
# print("Your private key is (d,n) ", private)

# message = input("Enter the message: ")

# message=message.replace(" ","")
# message=message.lower()
# arr=[]
# cipher_text=[]
# for i in message:
#     if i in main:
#         arr.append(main.index(i))
# for i in arr:
#     cipher_text.append(encrypt(public,i))

# print("Encrypted message (Cipher Text): ",cipher_text)

# plain=[]
# for i in cipher_text:
#     plain.append(decrypt(private,i))
# plain_text=''
# for i in plain:
#     plain_text=plain_text+main[i]
# print("Plain text array: ",plain)
# print("Decrypted message (Plain Text): ", plain_text)


'''
----------OUTPUT----------
Enter prime p: 157
Enter prime q (!=p): 149
Prime number p:  157
Prime number q:  149
Generating Public/Private key-pairs!
Value of n:  23393
Value of phi(n):  23088
Enter e such that is co-prime to  23088 : 
23
Value of exponent(e) entered is:  23
Your public key is (e,n)  (23, 23393)
Your private key is (d,n)  (6023, 23393)
Enter the message: world winner
Encrypted message (Cipher Text):  [22344, 18436, 8290, 15143, 15339, 22344, 8926, 8780, 8780, 22321, 8290]
Plain text array:  [22, 14, 17, 11, 3, 22, 8, 13, 13, 4, 17]
Decrypted message (Plain Text):  worldwinner
>>> 
'''


'\n----------OUTPUT----------\nEnter prime p: 157\nEnter prime q (!=p): 149\nPrime number p:  157\nPrime number q:  149\nGenerating Public/Private key-pairs!\nValue of n:  23393\nValue of phi(n):  23088\nEnter e such that is co-prime to  23088 : \n23\nValue of exponent(e) entered is:  23\nYour public key is (e,n)  (23, 23393)\nYour private key is (d,n)  (6023, 23393)\nEnter the message: world winner\nEncrypted message (Cipher Text):  [22344, 18436, 8290, 15143, 15339, 22344, 8926, 8780, 8780, 22321, 8290]\nPlain text array:  [22, 14, 17, 11, 3, 22, 8, 13, 13, 4, 17]\nDecrypted message (Plain Text):  worldwinner\n>>> \n'

### 3. Using openssl for web server - browser communication 


https://www.geeksforgeeks.org/practical-uses-of-openssl-command-in-linux/

OpenSSL is a cryptography software library or toolkit that makes communication over computer networks more secure. The OpenSSL program is a command-line tool for using the various cryptography functions of OpenSSL’s crypto library from the shell. It is generally used for Transport Layer Security(TSL) or Secure Socket Layer(SSL) protocols. OpenSSL is licensed under an apache-style license, which means that under some simple license conditions, one can use the toolkit for commercial or non-commercial purposes.

You can check the installed version of OpenSSL command using the following command

$openssl version
Checking-Openssl-Version-in-Linux

#### Practical Uses of OpenSSL Command
##### 1. To Create RSA Private Key.

$openssl genrsa -out private.key 2048
It will generate the RSA key file with the name private.key. Here, we have used 2048 for high security. Lower bit size can even be used.

Creating-RSA-Private-Key-using-Openssl-command-in-Linux

##### 2. Create new Private Key and CSR.

$openssl req -nodes -newkey rsa:2048 -keyout custom.key -out custom.csr
It will ask for the details like country code, state and locality name, Organization name, your name, email address, etc. And after entering all the details it will generate 2 files one with the CSR extension and the other with key extension representing CSR and private key respectively.

Creating-New-Private-Key-and-CSR-using-Openssl-command-in-Linux

##### 3. Create new Private Key and Self Signed certificate.

$openssl req -x509 -sha512 -nodes -days 730 -newkey rsa:2048 -keyout custom.key -out custom.pem
It will ask for details like country code, state and locality name, Organization name, your name, email address, etc. And after entering all the details it will generate 2 files one with the PEM extension and the other with key extension representing Self Signed Certificate and private key respectively. In the example, we have set validity to 730 days but in case you don’t mention this then it will take the value of one month by default. You can even change the algorithm of encryption as per your own convenience. In this example, we have used the SHA512 algorithm.

Creating-New-Private-Key-and-Self-Signed-Certificate-using-Openssl-Commad-in-Linux

##### 4. Verifying a CSR file

$openssl req -noout -text -in custom.csr
It will display the details you entered at the time of creating the CSR file which could be used to verify that the correct CSR file is sent to the correct receiver.

Verifying-a-CSR-File-using-OpenSSL-command-in-Linux

##### 5. Verifying a private key file.

$openssl rsa -in private.key -check
It will verify and check the RSA key and if it is Ok it will display the following result.

Verifying-a-Private-Key-File-using-openssl-command-in-Linux

##### 6. Verifying the Certificate Signer Authority

$openssl x509 -in custom.pem -noout -issuer -issuer_hash
It will display the details you entered at the time of creating the pem file which could be used to verify that the correct pem file is sent to the correct receiver.

Verifying-the-Certificate-Signer-Authority-using-Openssl-command-in-Linux

##### 7. Checking Hash value of a certificate

$openssl x509 -noout -hash -in custom.pem
It will display the hash value of the pem certificate file.

Checking-Hash-Value-of-a-Certificate-using-openssl-command-in-Linux

##### 8. Converting PEM to DER format

$openssl x509 -outform der -in custom.pem -out custom.der
It will change the extension of the certificate from .pem to .der and will create a new file with .der extension.

Converting-PEM-to-DER-format-using-openssl-command-in-Linux

##### 9. Checking pem file certificate expiry date

$openssl x509 -noout -in custom.pem -dates
It will display the valid from and valid up to date of the certificate.

Checking-pem-file-certificate-expiry-date-using-openssl-command-in-Linux

### 4. Using GNU PGP 
https://www.geeksforgeeks.org/encrypting-and-decrypting-the-files-using-gnupg-in-linux/

GnuPG is an encryption module that uses OpenPGP at its core. PGP stands for Pretty Good Privacy. It is an encryption program that provides authentication and cryptographic privacy for data communication. In the age where data is the new oil, the modern age thieves won’t intrude in through doors, windows or roofs, but instead, via wires and electrical signals in form of a few lines of code and commands. Nothing in this world is secure, which leads to an obvious conclusion that these thieves are inevitable. 

But instead of trying to figure out which door these thieves will intrude through, we might just need to focus on what is it they need … Data. Data is the holy grail of wine which contains the ingredients of an individual’s social, financial, emotional, habitual and sometimes physical well-being. Encryption can serve as a solution to elude hackers, private organizations and government surveillance systems from monitoring your data. Encryption makes data useless to the person who does not possess a decryption key and useful to the one who does. If the key is lost, the data create remains locked forever. 

#### Using GnuPG through Linux Terminal
We are assuming that you have installed GnuPG already. If not, go to GnuPG official website and download the required software and install it.  

Downloading-GnuPG

##### Encryption Process: 

1. You can start the encryption process by generating a key.  

gpg --gen-key 
2. A name and an email address will later serve you like an easy way to remember your key rather than the long key-id. 
 

Generating-Key-For-Encryption-Process

3. As soon as you choose okay a prompt for entering a passphrase will popup. This passphrase serves as a password for confirmation of the decryption key. It is a last line of defense for verification that the key belongs to you. 

4. To see all the keys in your keyring, you can use the below command 

$gpg --list-keys
5. Considering a file named sample.txt which is to be encrypted. For encrypting, use the below command  

$gpg --output encryptionoutput.gpg --encrypt --recipient geek@geeksforgeeks.org sample.txt
Here, .gpg is an extension used for encrypted files. 

##### Decryption Process: 

1. The encrypted file named encryptionoutput.gpg from the above given encryption process is to be sent to the recipient and in case you being the recipient, decryption is needed. Use the below command:  

$gpg --output decrypted_sample.txt --decrypt encryptionoutput.gpg
2. Now it will ask you to enter the passphrase to unlock the OpenPGP secret key.  

Decrypting-File-in-Linux-1

Now, you can check the result using ls command. The decryption process will only work if the private key has been imported into the keyring. You can also confirm the decryption by reading the content of the original file and decrypted file.  

Output-of-Decryption

Important Points:  

Passphrase is needed for decryption, importing/exporting of private keys.
GnuPG is user-specific as all other terminals programs i.e. if you run gpg as root then gpg will refer the root user keyring for further processes.
Losing the private keys will result in a total loss of data encrypted via those keys and recovery will be impossible until the Quantum computers arrive and make the decryption math a child’s play.

### 5. Perfordce evaluation of various cryptographic algorithms 

[1.png here](./1.png)

[2.png here](./2.png)


AES, DES, 3DES, RC6, Blowfish and RC2. Several points can be concluded from the simulation results. First; there is no significant difference when the results are displayed either in hexadecimal base encoding or in base 64 encoding. Secondly; in the case of changing packet size, it was concluded that Blowfish has better performance than other common encryption algorithms used, followed by RC6. Third; in the case of changing data type such as image instead of text, it was found that RC2, RC6 and Blowfish has disadvantage over other algorithms in terms of time consumption. Also, we find that 3DES still has low performance compared to algorithm DES. Finally -in the case of changing key size – it can be seen that higher key size leads to clear change in the battery and time consumption. 

// 6. Using IPTABLES on Linux and setting the filtering rules 


### 7. Configuring S/MIME for e-mail communication 

https://help.sap.com/doc/saphelp_nw75/7.5.5/en-US/be/93a0202c25482eadb02fc51081213c/content.htm?no_cache=true

To send and/or receive encrypted e-mails with S/MIME, you must exchange the e-mail certificates between the AS ABAP server and the communication partner.

Prerequisites
If you only want to send encrypted e-mails, you can ignore the prerequisites. If, however, you want to receive encrypted e-mails, you must fulfill the following prerequisites:

You have created S/MIME identities in the trust manager.

You have imported the required CA certificates and PSEs. In Configuring E-Mails with S/MIME (AS ABAP E-Mail Server), you find more information about the creation of S/MIME PSEs with a trust anchor.

Context
For sending and/or receiving encrypted e-mails with S/MIME, there are several options of exchanging e-mail certificates between the AS ABAP server and the communication partner:

Sending signed e-mails to one another. By default, a signed e-mail already includes the encryption certificate.

Manual import

Procedure
Option 1: When you and your communication partner send signed e-mails to one another, the AS ABAP automatically imports the encryption certificate to its address book.

Option 2: To manually import the encryption certificate, perform the following steps:

Start the trust manager (transaction STRUST).
Choose Start of the navigation path Certificate Next navigation step Import End of the navigation path.
Select the File tab.
Choose the certificate file in the relevant path.
Choose Open.
Choose GUI_ICON_SYSTEM_BACK(Input). The content of the certificate is now displayed in the Certificate section.
Choose Start of the navigation path Certificate Next navigation step Export End of the navigation path.
Select the tab for the address book.
Choose GUI_ICON_SYSTEM_BACK(Input). This includes your certificate in the address book.
Option 3: The SMIME enhancement spot contains the SMIME_EMAIL BAdI, which enables you to influence the certificate retrieval and selection process:


You need the certificate of a communication partner’s e-mail address that is not stored in the address book of the trust manager. In this case, you derive your own implementation class from the default implementation class of this BAdI. You overwrite/redefine the CERTIFICATE_RETRIEVAL method with your own implementation to find a certificate that is associated with an e-mail address of the communication partner. For example, an LDAP server can provide this e-mail address.

When you implement the BAdI method CERTIFICATE_SELECTION, you can resolve ambiguity concerning certificate usage. This occurs if there are several valid certificates for the same e-mail address. The default implementation checks whether the period of validity of a certificate might have expired, a CRL might prevent you from using it, or the key usage has the wrong type.

For more information, see the system documentation in the SMIME enhancement spot in Enhancements (transaction SE20), and the relevant BAdI methods in interface IF_BADI_SMIME_EMAIL and in the default implementation class CL_SMIME_EMAIL_BADI_DEFAULT.


### 8. Understanding the buffer overflow and format string attacks 

#### Buffer overflow
In information security and programming, a buffer overflow, or buffer overrun, is an anomaly where a program, while writing data to a buffer, overruns the buffer's boundary and overwrites adjacent memory locations.

Buffers are areas of memory set aside to hold data, often while moving it from one section of a program to another, or between programs. Buffer overflows can often be triggered by malformed inputs; if one assumes all inputs will be smaller than a certain size and the buffer is created to be that size, then an anomalous transaction that produces more data could cause it to write past the end of the buffer. If this overwrites adjacent data or executable code, this may result in erratic program behavior, including memory access errors, incorrect results, and crashes.

Programming languages commonly associated with buffer overflows include C and C++, which provide no built-in protection against accessing or overwriting data in any part of memory and do not automatically check that data written to an array (the built-in buffer type) is within the boundaries of that array. Bounds checking can prevent buffer overflows, but requires additional code and processing time. Modern operating systems use a variety of techniques to combat malicious buffer overflows, notably by randomizing the layout of memory, or deliberately leaving space between buffers and looking for actions that write into those areas ("canaries").

```
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
 
void vulnerableFunc(char* input) {
    char buffer[80];
    strcpy(buffer, input);
}
 
int main(int argc, char** argv) {
    if (argc != 2) {
        printf("Arguments: <buffer input>\n");
        exit(1);
    }
 
    vulnerableFunc(argv[1]);
 
    printf("Exiting...\n");
    exit(0);
}
```

##### normal :
./overflow AAAAAAAAAAAAAAAAAAAAAAAAAAA
Exiting....

##### Vulnerable
./overflow AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

segmentation fault

####  format string attack
Uncontrolled format string is a type of software vulnerability discovered around 1989 that can be used in security exploits.[1] Originally thought harmless, format string exploits can be used to crash a program or to execute harmful code. The problem stems from the use of unchecked user input as the format string parameter in certain C functions that perform formatting, such as printf(). A malicious user may use the %s and %x format tokens, among others, to print data from the call stack or possibly other locations in memory. One may also write arbitrary data to arbitrary locations using the %n format token, which commands printf() and similar functions to write the number of bytes formatted to an address stored on the stack.

```
#include  <stdio.h> 
void main(int argc, char **argv)
{
	// This line is safe
	printf("%s\n", argv[1]);

	// This line is vulnerable
	printf(argv[1]);
}
```
##### Safe Code
The line printf("%s", argv[1]); in the example is safe, if you compile the program and run it:

./example "Hello World %s%s%s%s%s%s"

The printf in the first line will not interpret the “%s%s%s%s%s%s” in the input string, and the output will be: “Hello World %s%s%s%s%s%s”

##### Vulnerable Code
The line printf(argv[1]); in the example is vulnerable, if you compile the program and run it:

./example "Hello World %s%s%s%s%s%s"

The printf in the second line will interpret the %s%s%s%s%s%s in the input string as a reference to string pointers, so it will try to interpret every %s as a pointer to a string, starting from the location of the buffer (probably on the Stack). At some point, it will get to an invalid address, and attempting to access it will cause the program to crash.


### 9.  Using NMP for ports monitoring 

https://www.freecodecamp.org/news/what-is-nmap-and-how-to-use-it-a-tutorial-for-the-greatest-scanning-tool-of-all-time/

#### What is Nmap?
Nmap is short for Network Mapper. It is an open-source Linux command-line tool that is used to scan IP addresses and ports in a network and to detect installed applications.

Nmap allows network admins to find which devices are running on their network, discover open ports and services, and detect vulnerabilities.

Gordon Lyon (pseudonym Fyodor) wrote Nmap as a tool to help map an entire network easily and to find its open ports and services.

Nmap has become hugely popular, being featured in movies like The Matrix and the popular series Mr. Robot.

#### Why use Nmap?
There are a number of reasons why security pros prefer Nmap over other scanning tools.

First, Nmap helps you to quickly map out a network without sophisticated commands or configurations. It also supports simple commands (for example, to check if a host is up) and complex scripting through the Nmap scripting engine.

##### Other features of Nmap include:

Ability to quickly recognize all the devices including servers, routers, switches, mobile devices, etc on single or multiple networks.
Helps identify services running on a system including web servers, DNS servers, and other common applications. Nmap can also detect application versions with reasonable accuracy to help detect existing vulnerabilities.
Nmap can find information about the operating system running on devices. It can provide detailed information like OS versions, making it easier to plan additional approaches during penetration testing.
During security auditing and vulnerability scanning, you can use Nmap to attack systems using existing scripts from the Nmap Scripting Engine.
Nmap has a graphical user interface called Zenmap. It helps you develop visual mappings of a network for better usability and reporting.
#### Commands
Let's look at some Nmap commands. If you don't have Nmap installed, you can get it from here.

##### Basic scans
Scanning the list of active devices on a network is the first step in network mapping. There are two types of scans you can use for that:

Ping scan — Scans the list of devices up and running on a given subnet.
> nmap -sp 192.168.1.1/24

Scan a single host — Scans a single host for 1000 well-known ports. These ports are the ones used by popular services like SQL, SNTP, apache, and others.
> nmap scanme.nmap.org

##### 1-3
Stealth scan
Stealth scanning is performed by sending an SYN packet and analyzing the response. If SYN/ACK is received, it means the port is open, and you can open a TCP connection.

However, a stealth scan never completes the 3-way handshake, which makes it hard for the target to determine the scanning system.

> nmap -sS scanme.nmap.org

You can use the ‘-sS’ command to perform a stealth scan. Remember, stealth scanning is slower and not as aggressive as the other types of scanning, so you might have to wait a while to get a response.

Version scanning
Finding application versions is a crucial part in penetration testing.

It makes your life easier since you can find an existing vulnerability from the Common Vulnerabilities and Exploits (CVE) database for a particular version of the service. You can then use it to attack a machine using an exploitation tool like Metasploit.

> nmap -sV scanme.nmap.org

To do a version scan, use the ‘-sV’ command. Nmap will provide a list of services with its versions. Do keep in mind that version scans are not always 100% accurate, but it does take you one step closer to successfully getting into a system.

##### 1-5
OS Scanning
In addition to the services and their versions, Nmap can provide information about the underlying operating system using TCP/IP fingerprinting. Nmap will also try to find the system uptime during an OS scan.

> nmap -sV scanme.nmap.org

You can use the additional flags like osscan-limit to limit the search to a few expected targets. Nmap will display the confidence percentage for each OS guess.

Again, OS detection is not always accurate, but it goes a long way towards helping a pen tester get closer to their target.

##### 1-6
Aggressive Scanning
Nmap has an aggressive mode that enables OS detection, version detection, script scanning, and traceroute. You can use the -A argument to perform an aggressive scan.

> nmap -A scanme.nmap.org

Aggressive scans provide far better information than regular scans. However, an aggressive scan also sends out more probes, and it is more likely to be detected during security audits.

##### 1-7
Scanning Multiple Hosts
Nmap has the capability of scanning multiple hosts simultaneously. This feature comes in real handy when you are managing vast network infrastructure.

You can scan multiple hosts through numerous approaches:

Write all the IP addresses in a single row to scan all of the hosts at the same time.
> nmap 192.164.1.1 192.164.0.2 192.164.0.2

Use the asterisk (*) to scan all of the subnets at once.
> nmap 192.164.1.*

Add commas to separate the addresses endings instead of typing the entire domains.
> nmap 192.164.0.1,2,3,4

Use a hyphen to specify a range of IP addresses
> nmap 192.164.0.0–255

#### Port Scanning
Port scanning is one of the most fundamental features of Nmap. You can scan for ports in several ways.

Using the -p param to scan for a single port
> nmap -p 973 192.164.0.1

If you specify the type of port, you can scan for information about a particular type of connection, for example for a TCP connection.
> nmap -p T:7777, 973 192.164.0.1

A range of ports can be scanned by separating them with a hyphen.
> nmap -p 76–973 192.164.0.1

You can also use the -top-ports flag to specify the top n ports to scan.
> nmap --top-ports 10 scanme.nmap.org

Scanning from a File
If you want to scan a large list of IP addresses, you can do it by importing a file with the list of IP addresses.

> nmap -iL /input_ips.txt

The above command will produce the scan results of all the given domains in the “input_ips.txt” file. Other than simply scanning the IP addresses, you can use additional options and flags as well.

#### Verbosity and Exporting Scan Results
Penetration testing can last days or even weeks. Exporting Nmap results can be useful to avoid redundant work and to help with creating final reports. Let’s look at some ways to export Nmap scan results.

Verbose Output
> nmap -v scanme.nmap.org

The verbose output provides additional information about the scan being performed. It is useful to monitor step by step actions Nmap performs on a network, especially if you are an outsider scanning a client’s network.

##### 1-8
Normal output
Nmap scans can also be exported to a text file. It will be slightly different from the original command line output, but it will capture all the essential scan results.

> nmap -oN output.txt scanme.nmap.org

##### 1-9
XML output
Nmap scans can also be exported to XML. It is also the preferred file format of most pen-testing tools, making it easily parsable when importing scan results.

> nmap -oX output.xml scanme.nmap.org

##### 1-10
Multiple Formats
You can also export the scan results in all the available formats at once using the -oA command.

> nmap -oA output scanme.nmap.org

The above command will export the scan result in three files — output.xml, output. Nmap and output.gnmap.

##### Nmap Help
Nmap has a built-in help command that lists all the flags and options you can use. It is often handy given the number of command-line arguments Nmap comes with.

> nmap -h

##### 1-12
Nmap Scripting Engine
Nmap Scripting Engine (NSE) is an incredibly powerful tool that you can use to write scripts and automate numerous networking features.

You can find plenty of scripts distributed across Nmap, or write your own script based on your requirements. You can even modify existing scripts using the Lua programming language.

##### 1-13
NSE also has attack scripts that are used in attacking the network and various networking protocols.

10. Implementation of proxy based security protocols in C or C++ with features like confidentiality, integrity and authentication 

